In [33]:
# Install pyDOE3 using pip
!pip install pyDOE3

In [35]:
# Import the central composite design function
from pyDOE3 import ccdesign
import numpy as np
import pandas as pd

## Create a basic CCD

In [ ]:
### Basic syntax

# ccdesign(n, center, alpha, face)

    # n = number of factors 
    # center = number of center points (tuple)
    # alpha = whether the design is orthogonal or rotatable ("o" or "r")
    # face = orientation of the design ("ccc"=circumscribed, "cci"=inscribed, "ccf"=faced)

In [37]:
# Create a central composite design for 2 factors
design = ccdesign(2)  
print(design) 

[[-1.         -1.        ]
 [-1.          1.        ]
 [ 1.         -1.        ]
 [ 1.          1.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [-1.41421356  0.        ]
 [ 1.41421356  0.        ]
 [ 0.         -1.41421356]
 [ 0.          1.41421356]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]]


## Customize design

In [45]:
# Customize center points (factorial, axial)
custom_center = ccdesign(2, center=(2, 2), face="ccf", alpha="r")  # This sets 2 center points for factorial portion and 2 for axial portion
print("Custom center points:")
print(custom_center) 

Custom center points:
[[-1. -1.]
 [-1.  1.]
 [ 1. -1.]
 [ 1.  1.]
 [ 0.  0.]
 [ 0.  0.]
 [-1.  0.]
 [ 1.  0.]
 [ 0. -1.]
 [ 0.  1.]
 [ 0.  0.]
 [ 0.  0.]]


## Add factor names

In [47]:
# Customize center points (factorial, axial)
design = ccdesign(2, center=(2, 2), alpha="r", face="ccc") 
# Convert to DataFrame with labels
df = pd.DataFrame(design, columns=['Temperature', 'Pressure'])  # Give meaningful names to our factors
print(df)

    Temperature  Pressure
0     -1.000000 -1.000000
1     -1.000000  1.000000
2      1.000000 -1.000000
3      1.000000  1.000000
4      0.000000  0.000000
5      0.000000  0.000000
6     -1.414214  0.000000
7      1.414214  0.000000
8      0.000000 -1.414214
9      0.000000  1.414214
10     0.000000  0.000000
11     0.000000  0.000000


## Scale to actual experimental levels

In [ ]:
def scale_design(design, factor_ranges):
    """
    Scale coded design to actual experimental values

    Parameters:
    -----------
    design : array
        The coded design matrix
    factor_ranges : list of tuples
        List of (min, max) values for each factor

    Returns:
    --------
    scaled_design : DataFrame
        The design with actual experimental values
    """
    scaled_design = design.copy()

    for i, (low, high) in enumerate(factor_ranges):
        # Scale the column
        center = (high + low) / 2
        range_val = high - low

        # Handle CCC designs where axial points extend beyond [-1, 1]
        max_abs_val = max(abs(design.iloc[:, i].min()), abs(design.iloc[:, i].max()))

        if max_abs_val > 1:
            # For values within [-1, 1]
            mask_within = (design.iloc[:, i] >= -1) & (design.iloc[:, i] <= 1)
            scaled_design.loc[mask_within, design.columns[i]] = center + design.loc[mask_within, design.columns[i]] * range_val / 2

            # For axial points
            mask_axial = ~mask_within
            scaled_design.loc[mask_axial, design.columns[i]] = center + design.loc[mask_axial, design.columns[i]] * range_val / 2
        else:
            # For CCF designs
            scaled_design.iloc[:, i] = center + design.iloc[:, i] * range_val / 2

    return scaled_design

In [49]:
# Customize center points (factorial, axial)
design = ccdesign(2, center=(2, 2), alpha="r", face="ccc")

# Convert to DataFrame with labels
df = pd.DataFrame(design, columns=['Temperature', 'Pressure'])  # Give meaningful names to our factors

# Define actual ranges for our factors
factor_ranges = [
    (60, 100),   # Temperature range (°C)
    (1, 5)       # Pressure range (bar)
]

# Scale our design
scaled_df = scale_design(df, factor_ranges)

print(scaled_df)

    Temperature  Pressure
0     60.000000  1.000000
1     60.000000  5.000000
2    100.000000  1.000000
3    100.000000  5.000000
4     80.000000  3.000000
5     80.000000  3.000000
6     51.715729  3.000000
7    108.284271  3.000000
8     80.000000  0.171573
9     80.000000  5.828427
10    80.000000  3.000000
11    80.000000  3.000000


In [51]:
# Export to Excel
scaled_df.to_excel('central_composite_design_raw.xlsx', index=True)  # Save our design for use in the lab